In [1]:
import matplotlib

matplotlib.rc('font', family='NanumBarunGothic')
matplotlib.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os.path

if not os.path.isfile('data/aclImdb_v1.tar.gz'):
    !wget -q http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz -P data
    !tar -xzf data/aclImdb_v1.tar.gz -C data

In [3]:
import tarfile

ap = tarfile.open("./data/aclImdb_v1.tar.gz")
ap.extractall("./data")
ap.close()

In [4]:
import shutil
shutil.rmtree('data/aclImdb/train/unsup')

In [13]:
from sklearn.datasets import load_files

reviews_train = load_files("data/aclImdb/train/")
text_train, y_train = reviews_train.data, reviews_train.target
type(text_train), len(text_train), text_train[1]

(list,
 25000,
 b'Words can\'t describe how bad this movie is. I can\'t explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do that. There are so many clich\xc3\xa9s, mistakes (and all other negative things you can imagine) here that will just make you cry. To start with the technical first, there are a LOT of mistakes regarding the airplane. I won\'t list them here, but just mention the coloring of the plane. They didn\'t even manage to show an airliner in the colors of a fictional airline, but instead used a 747 painted in the original Boeing livery. Very bad. The plot is stupid and has been done many times before, only much, much better. There are so many ridiculous moments here that i lost count of it really early. Also, I was on the bad guys\' side all the time in the movie, because the good guys were so stupid. "Executive Decision" should without a doubt be you\'re choice over this one, eve

In [14]:
type(text_train[0])

bytes

In [15]:
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

In [16]:
import numpy as np
np.bincount(y_train)

array([12500, 12500], dtype=int64)

In [17]:
reviews_test = load_files("data/aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target

In [10]:
len(text_test)

25000

In [11]:
np.bincount(y_test)

array([12500, 12500], dtype=int64)

## BOW ( bag of words )

In [12]:
bards_words = ["The fool doth think he is wise,",
              "but the wise man knows himself to be a fool"]

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(bards_words)

CountVectorizer()

In [14]:
len(vect.vocabulary_)

13

In [15]:
vect.vocabulary_

{'the': 9,
 'fool': 3,
 'doth': 2,
 'think': 10,
 'he': 4,
 'is': 6,
 'wise': 12,
 'but': 1,
 'man': 8,
 'knows': 7,
 'himself': 5,
 'to': 11,
 'be': 0}

In [16]:
bag_of_words = vect.transform(bards_words)
bag_of_words.toarray()

array([[0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1],
       [1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1]], dtype=int64)

In [17]:
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
X_train

<25000x74849 sparse matrix of type '<class 'numpy.int64'>'
	with 3431196 stored elements in Compressed Sparse Row format>

In [18]:
len(vect.vocabulary_)

74849

In [19]:
feature_names = vect.get_feature_names_out()
len(feature_names)

74849

In [20]:
feature_names[2000:2010]

array(['aesir', 'aesop', 'aestheically', 'aesthete', 'aesthetic',
       'aesthetical', 'aesthetically', 'aestheticism', 'aesthetics',
       'aetheist'], dtype=object)

In [21]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

scores = cross_val_score(LogisticRegression(max_iter=1000), X_train, y_train, n_jobs=-1)
np.mean(scores)

0.88124

In [22]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter=5000), param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_score_, grid.best_params_

(0.8881599999999998, {'C': 0.1})

In [23]:
X_test = vect.transform(text_test)
grid.score(X_test, y_test)

0.87892

In [24]:
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
X_train

<25000x27271 sparse matrix of type '<class 'numpy.int64'>'
	with 3354014 stored elements in Compressed Sparse Row format>

In [25]:
feature_names = vect.get_feature_names_out()

In [26]:
feature_names[:50]

array(['00', '000', '007', '00s', '01', '02', '03', '04', '05', '06',
       '07', '08', '09', '10', '100', '1000', '100th', '101', '102',
       '103', '104', '105', '107', '108', '10s', '10th', '11', '110',
       '112', '116', '117', '11th', '12', '120', '12th', '13', '135',
       '13th', '14', '140', '14th', '15', '150', '15th', '16', '160',
       '1600', '16mm', '16s', '16th'], dtype=object)

In [27]:
feature_names[20010:20030]

array(['repentance', 'repercussions', 'repertoire', 'repetition',
       'repetitions', 'repetitious', 'repetitive', 'rephrase', 'replace',
       'replaced', 'replacement', 'replaces', 'replacing', 'replay',
       'replayable', 'replayed', 'replaying', 'replays', 'replete',
       'replica'], dtype=object)

In [28]:
feature_names[-2:]

array(['zwick', 'émigré'], dtype=object)

In [29]:
grid = GridSearchCV(LogisticRegression(max_iter=5000), param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_score_, grid.best_params_

(0.88812, {'C': 0.1})

## 불용어

In [30]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
len(ENGLISH_STOP_WORDS), list(ENGLISH_STOP_WORDS)[::10]

(318,
 ['are',
  'without',
  'along',
  'third',
  'amongst',
  'thence',
  'may',
  'it',
  'by',
  'four',
  'onto',
  'seem',
  'toward',
  'take',
  'us',
  'done',
  'below',
  'of',
  'part',
  'indeed',
  'eleven',
  'your',
  'once',
  'whereas',
  'thereby',
  'eg',
  'few',
  'was',
  'sometimes',
  'get',
  'wherein',
  'formerly'])

In [31]:
vect = CountVectorizer(min_df=5, stop_words="english").fit(text_train)
X_train = vect.transform(text_train)
X_train

<25000x26966 sparse matrix of type '<class 'numpy.int64'>'
	with 2149958 stored elements in Compressed Sparse Row format>

In [32]:
grid = GridSearchCV(LogisticRegression(max_iter=5000), param_grid, n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_score_, grid.best_params_

(0.8828400000000001, {'C': 0.1})

## tf-idf

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression(max_iter=5000))
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}

grid = GridSearchCV(pipe, param_grid, n_jobs=-1)
grid.fit(text_train, y_train)
grid.best_score_, grid.best_params_

(0.89192, {'logisticregression__C': 10})

In [34]:
vectorizer = grid.best_estimator_.named_steps['tfidfvectorizer']
X_train = vectorizer.transform(text_train)
max_value = X_train.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
feature_names = np.array(vectorizer.get_feature_names_out())
feature_names[sorted_by_tfidf[:20]]

array(['suplexes', 'gauche', 'hypocrites', 'oncoming', 'songwriting',
       'galadriel', 'emerald', 'mclaughlin', 'sylvain', 'oversee',
       'cataclysmic', 'pressuring', 'uphold', 'thieving', 'inconsiderate',
       'ware', 'denim', 'reverting', 'booed', 'spacious'], dtype=object)

In [35]:
feature_names[sorted_by_tfidf[-20:]]

array(['gadget', 'sucks', 'zatoichi', 'demons', 'lennon', 'bye', 'dev',
       'weller', 'sasquatch', 'botched', 'xica', 'darkman', 'woo',
       'casper', 'doodlebops', 'smallville', 'wei', 'scanners', 'steve',
       'pokemon'], dtype=object)

In [36]:
sorted_by_idf = np.argsort(vectorizer.idf_)
feature_names[sorted_by_idf[:100]]

array(['the', 'and', 'of', 'to', 'this', 'is', 'it', 'in', 'that', 'but',
       'for', 'with', 'was', 'as', 'on', 'movie', 'not', 'have', 'one',
       'be', 'film', 'are', 'you', 'all', 'at', 'an', 'by', 'so', 'from',
       'like', 'who', 'they', 'there', 'if', 'his', 'out', 'just',
       'about', 'he', 'or', 'has', 'what', 'some', 'good', 'can', 'more',
       'when', 'time', 'up', 'very', 'even', 'only', 'no', 'would', 'my',
       'see', 'really', 'story', 'which', 'well', 'had', 'me', 'than',
       'much', 'their', 'get', 'were', 'other', 'been', 'do', 'most',
       'don', 'her', 'also', 'into', 'first', 'made', 'how', 'great',
       'because', 'will', 'people', 'make', 'way', 'could', 'we', 'bad',
       'after', 'any', 'too', 'then', 'them', 'she', 'watch', 'think',
       'acting', 'movies', 'seen', 'its', 'him'], dtype=object)

## 여러 단어로 만든 BOW(n-그램)

In [38]:
bards_words

['The fool doth think he is wise,',
 'but the wise man knows himself to be a fool']

In [40]:
cv = CountVectorizer(ngram_range=(1, 1)).fit(bards_words)
len(cv.vocabulary_), cv.get_feature_names_out()

(13,
 array(['be', 'but', 'doth', 'fool', 'he', 'himself', 'is', 'knows', 'man',
        'the', 'think', 'to', 'wise'], dtype=object))

In [42]:
cv = CountVectorizer(ngram_range=(2, 2)).fit(bards_words)
len(cv.vocabulary_), cv.get_feature_names_out()

(14,
 array(['be fool', 'but the', 'doth think', 'fool doth', 'he is',
        'himself to', 'is wise', 'knows himself', 'man knows', 'the fool',
        'the wise', 'think he', 'to be', 'wise man'], dtype=object))

In [43]:
cv = CountVectorizer(ngram_range=(1, 2)).fit(bards_words)
len(cv.vocabulary_), cv.get_feature_names_out()

(27,
 array(['be', 'be fool', 'but', 'but the', 'doth', 'doth think', 'fool',
        'fool doth', 'he', 'he is', 'himself', 'himself to', 'is',
        'is wise', 'knows', 'knows himself', 'man', 'man knows', 'the',
        'the fool', 'the wise', 'think', 'think he', 'to', 'to be', 'wise',
        'wise man'], dtype=object))

In [44]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression(max_iter=5000))
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100], 
             'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)]}

grid = GridSearchCV(pipe, param_grid, n_jobs=-1)
grid.fit(text_train, y_train)
grid.best_score_, grid.best_params_

(0.9064400000000001,
 {'logisticregression__C': 100, 'tfidfvectorizer__ngram_range': (1, 3)})

## 어간추출 표제어추출

In [1]:
import spacy
import nltk

spacy.__version__, nltk.__version__

('2.3.5', '3.7')

In [8]:
en_nlp = spacy.load('en_core_web_sm')
stemmer = nltk.stem.PorterStemmer()

def compare_normalization(doc):
    doc_spacy = en_nlp(doc)
    print("표제어: ")
    print([token.lemma_ for token in doc_spacy])
    print("어간:")
    print([stemmer.stem(token.norm_.lower()) for token in doc_spacy])

In [9]:
compare_normalization(u"Our meeting today was worse than yesterday, "
                       "I'm scared of meeting the clients tomorrow.")

표제어: 
['-PRON-', 'meeting', 'today', 'be', 'bad', 'than', 'yesterday', ',', '-PRON-', 'be', 'scared', 'of', 'meet', 'the', 'client', 'tomorrow', '.']
어간:
['our', 'meet', 'today', 'wa', 'wors', 'than', 'yesterday', ',', 'i', 'am', 'scare', 'of', 'meet', 'the', 'client', 'tomorrow', '.']


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
en_nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def custom_tokenizer(document):
    doc_spacy = en_nlp(document)
    return [token.lemma_ for token in doc_spacy]

lemma_vect = CountVectorizer(tokenizer=custom_tokenizer, min_df=5)

In [18]:
X_train_lemma = lemma_vect.fit_transform(text_train)
X_train_lemma.shape

(25000, 22100)

In [19]:
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
X_train.shape

(25000, 27271)

In [22]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.99,
                           train_size=0.01, random_state=0)
grid = GridSearchCV(LogisticRegression(max_iter=5000), param_grid, cv=cv, n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_score_

0.7187717171717172

In [23]:
grid.fit(X_train_lemma, y_train)
grid.best_score_

0.7187151515151514